In [103]:
from __future__ import division, print_function
import requests
import time
from bs4 import BeautifulSoup

import pandas as pd
import re
from pprint import pprint

import pickle

import numpy as np

import wikipedia
import string

import datetime

#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')

In [74]:
#Use soup to get wiki artists pages

artist_urls = []
for char in string.ascii_uppercase:
    url = 'https://en.wikipedia.org/wiki/List_of_painters_by_name_beginning_with_%22' + char + '%22'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    ul = soup.find('ul')
    artists = ul.find_all('li')
    for artist in artists:
        link = artist.find('a')
        if link:
            artist_urls.append(link.get('href'))


In [93]:
artist_urls = [url for url in set(artist_urls) if (not 'redlink' in url) and (not 'commons.wikimedia' in url)]
len(artist_urls)


3179

In [ ]:
#Scrape wiki htmls for artist name and page id.

#page_id = {}

for artist_url in artist_urls[2770:]:
    if len(page_id.keys())%100 == 0:
        print(artist_url, datetime.datetime.now())
    url = 'https://en.wikipedia.org' + artist_url
    response = requests.get(url)
    
    page = response.text
    
    if not 'All article disambiguation pages' in page:
        start = page.find('wgArticleId') + 13
        end = start + page[start:].find(',')
        page_id[artist_url] = page[start:end]

In [94]:
output = open('wiki_pageid.pkl', 'wb')
pickle.dump(artist_urls, output)
output.close;

'https://commons.wikimedia.org/wiki/Dirck_de_Quade_van_Ravesteyn'

In [95]:
#Use wikipedia api to get content, name and links
data = {
    'wiki_pageid' : [],
    'wiki_url': [],
    'wiki_artist_name': [],
    'wiki_content': [],
    'wiki_links': [],
}
for artist_url in page_id:
    if len(data['wiki_links'])%100 == 0:
        print(artist_url, datetime.datetime.now())
    artist_page = wikipedia.page(pageid=page_id[artist_url])
    data['wiki_url'].append(artist_url)
    data['wiki_artist_name'].append(artist_page.title)
    data['wiki_content'].append(artist_page.content)
    data['wiki_links'].append(artist_page.links)
    data['wiki_pageid'].append(page_id[artist_url])

In [100]:
len(data['wiki_artist_name'])

3178

In [114]:
#Create Links Matrix
artist_names = [name for name in data['wiki_artist_name']]
n_artist = len(artist_names)
G = np.zeros([n_artist,n_artist])
for i in range(n_artist):
    for link in data['wiki_links'][i]:
        for j, name in enumerate(artist_names):
            if name == link:
                G[i,j] = 1


In [117]:
print(G.sum()) 
for i in range(n_artist):
    G[i,i] = 0
print(G.sum())    

14992.0
14962.0


In [126]:
output = open('wikiartist_network.pkl', 'wb')
pickle.dump(G, output)
output.close;

In [119]:
import pagerank2
pagerank = list(pagerank2.powerIteration(G))
data['pagerank'] = n_artist

0
10
20


In [124]:
df['pagerank'] = n_artist*df.pagerank

In [125]:
df.sort_values('pagerank', ascending=False)

,pagerank,wiki_artist_name,wiki_content,wiki_links,wiki_pageid,wiki_url
593,23.507505,Rembrandt,Rembrandt Harmenszoon van Rijn (; Dutch: [ˈrɛm...,"[19th-century French art, A Polish Nobleman, A...",4254144,/wiki/Rembrandt_Harmenszoon_van_Rijn
2303,22.790955,Raphael,Raffaello Sanzio da Urbino (Italian: [raffaˈɛl...,"[Academic art, Agostino Chigi, Alba Madonna, A...",44525,/wiki/Raffaello_Santi
3006,20.556661,Leonardo da Vinci,Leonardo di ser Piero da Vinci (Italian: [leoˈ...,"[A Bird in Flight, A Mathematician's Apology, ...",18079,/wiki/Leonardo_da_Vinci
735,20.210639,Pablo Picasso,Pablo Picasso (; Spanish: [ˈpaβlo piˈkaso]; 25...,"[20th-century art, 3rd Sculpture International...",24176,/wiki/Pablo_Picasso
1302,19.997338,Michelangelo,Michelangelo di Lodovico Buonarroti Simoni (; ...,"[Adam and Eve, Agostino di Duccio, Anatomical,...",21019,/wiki/Michelangelo_Buonarroti
2103,17.613982,Titian,Tiziano Vecelli or Tiziano Vecellio (pronounce...,"[A Man with a Quilted Sleeve, Alfonso I d'Este...",154239,/wiki/Titian
3150,16.720815,Paul Cézanne,Paul Cézanne (US: or UK: ; French: [pɔl sezan...,"[Académie Suisse, Académie des Beaux-Arts, Ach...",24472,/wiki/Paul_C%C3%A9zanne
618,15.771616,Peter Paul Rubens,Sir Peter Paul Rubens (; Dutch: [ˈrybə(n)s]; 2...,"[A Bearded Man, A Dog of Flanders, A View of H...",21463370,/wiki/Peter_Paul_Rubens
1217,13.415132,Vincent van Gogh,Vincent Willem van Gogh (Dutch: [ˈvɪnsɛnt ˈʋɪl...,"[87 Hackford Road (Van Gogh), ABC Online, AT5,...",32603,/wiki/Vincent_van_Gogh
1653,13.412863,Édouard Manet,Édouard Manet (US: or UK: ; French: [edwaʁ ma...,"[A Bar at the Folies-Bergère, Adolphe Thiers, ...",9615,/wiki/%C3%89douard_Manet


In [123]:
df = pd.DataFrame(data)
df.to_csv('wiki_scrape.csv', encoding='utf-8')

output = open('wiki_scrape.pkl', 'wb')
pickle.dump(df, output)
output.close;

In [ ]:
#Match with Artist on FindArtInfo.com
data1 = {}
data1['wiki_artist_name'] = []
data1['fai_links'] = []

for artist in artist_names:
    url = 'http://www.findartinfo.com/english/Artists/Result?artistName=' + artist
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    fai_links = []
    for tag in soup.find_all(class_ = 'linkgoogle'):
        link  = tag.find('a').get('href')
        if 'prices-by' in link:
            fai_links.append(link)
    fai_links = list(set(fai_links))
    data1['fai_links'].append(fai_links)
    data1['wiki_artist_name'].append(artist)
    

In [ ]:
data1

In [ ]:
#get mapping from country number to country

country_numbers = {}
url = 'http://www.findartinfo.com/english/advanced-search.html'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

for tag in soup.find_all('option'):
    number = tag.attrs['value']
    country_numbers[number] = tag.text

output = open('country_numbers.pkl', 'wb')

pickle.dump(country_numbers, output)
output.close;

In [ ]:
#Scrape artist pages for relevant auction sales

data2 = {}
data2['trans_date'] = []
data2['title'] = []
data2['size'] = []
data2['medium'] = []
data2['price'] = []
data2['fai_artist_name'] = []
data2['artist_born'] = []
data2['artist_died'] = []
data2['artist_country'] = []
data2['wiki_artist_name'] = []


count = 0
for links in data1['fai_links']:
    wan = data1['wiki_artist_name'][count]
    count += 1
    if count % 50 == 0:
        time.sleep(10)
        df = pd.DataFrame(data2)
        df.to_csv('fai_scrape.csv')
        print(count, len(data2['price']), link)
    for link in links:


        url = 'http://www.findartinfo.com' + link
        response = requests.get(url)
        if response.status_code != 200:
            print(response.status_code)
            break

        page = response.text
        soup = BeautifulSoup(page, "lxml")

        artist_name = soup.find('h1').text.strip().split()[4:]
        artist_name = ' '.join(artist_name)
        artist_born = (soup
                       .find('table', style="margin-top:5px")
                       .find_all('tr')[0]
                       .find_all('td')[1]
                       .text
                       .strip())
        artist_died = (soup
                       .find('table', style="margin-top:5px")
                       .find_all('tr')[1]
                       .find_all('td')[1]
                       .text
                       .strip())
        artist_country = (soup
                          .find('table', style="margin-top:5px")
                          .find_all('tr')[2]
                          .find_all('td')[1]
                          .text
                          .strip())

        pages = int(re.findall("Page 1 of \d*", str(soup.find('h2')))[0].split()[-1])
        for i in range(pages):
            url = 'http://www.findartinfo.com' + link[:-5] + '/page/' + str(i+1) + '.html'
            response = requests.get(url)
            page = response.text
            soup = BeautifulSoup(page, "lxml")
            rows = soup.find_all(onmouseover="this.style.backgroundColor = '#EEEEEE' ")
            for row in rows:
                row_data = row.find_all('td')
                data2['trans_date'].append(row_data[1].get_text().strip())
                data2['title'].append(row_data[2].get_text().strip())
                data2['size'].append(row_data[3].get_text().strip())
                data2['medium'].append(row_data[4].get_text().strip())
                data2['price'].append(row_data[5].get_text().strip())
                data2['fai_artist_name'].append(artist_name)
                data2['artist_born'].append(artist_born)
                data2['artist_died'].append(artist_died)
                data2['artist_country'].append(artist_country) 
                data2['wiki_artist_name'].append(wan)

In [ ]:
df = pd.DataFrame(data2)
df.to_csv('fai_scrape.csv', encoding='utf-8')

In [ ]:
df

# below are attempts not used in final estimation

## Use Google n-grams to get popularity

In [ ]:
from google_ngram import Gngram
import pandas as pd

In [ ]:
dfs = []
i = 0
while 5*i+5 < len(names):
    g = Gngram(names[5*i: 5*i+5], years=[1940, 2000], case_insensitive=True)
    dfs.append(g.df_parents)
    i += 1
    if i % 50 == 0:
        print(i)
g = Gngram(names[5*i:], years=[1940, 2000], case_insensitive=True)
dfs.append(g.df_parents)

In [ ]:
g.df_parents.mean()

In [ ]:
g = Gngram(names[: 5], years=[1940, 2000], case_insensitive=False)
g.df_parents.mean()

### get name probabilties from azure

In [ ]:
import requests
import json
import pandas as pd

import unicodedata

In [ ]:
df = pd.read_csv('paintings.csv', encoding='utf-8')
names_in = df.artist_name.unique()
names = []
for i in range(len(names_in)):
    name = names_in[i].split()
    if (len(name) >= 2) and ( not '.' in name[0]) :
        name_out = name[0] + ' ' + name[-1]
        (name[0] + ' ' + name[-1])
        name_out = unicodedata.normalize('NFKD', name_out).encode('ascii','ignore')
        names.append(name_out)
len(names)

In [ ]:
def get_prob(names):
    d1 = {'queries': names}

    url = 'https://westus.api.cognitive.microsoft.com/text/weblm/v1.0/calculateJointProbability?model=body'
    headers = {'Host': 'westus.api.cognitive.microsoft.com', 'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': '94f6353e97e9471b855034a04247675c'}
    data = json.dumps(d1)

    response = requests.post(url, headers=headers, data=data)
    
    #print(response.text)
    out = []
    for i in range(len(names)):
        out.append(json.loads(response.text)['results'][i]['probability'])
    return out


In [ ]:
data = {'artist_name': [], 'name_prob': []}
i = 0
while 100*i+ 100 < len(names):
    if i%10 == 0:
        print i
        df = pd.DataFrame(data)
        df.to_csv('Name_Probs.csv')
    probs = get_prob(names[i*100:i*100 +100])
    data['artist_name'].extend(names[i*100:i*100 +100])
    data['name_prob'].extend(probs)
    i+=1

probs = get_prob(names[i*100:])
data['artist_name'].extend(names[i*100:])
data['name_prob'].extend(probs)


df = pd.DataFrame(data)
df.to_csv('Name_Probs.csv')

In [ ]:
#add back original names
df = pd.read_csv('paintings.csv', encoding='utf-8')
names_in = df.artist_name.unique()
name_map = {}
for i in range(len(names_in)):
    name = names_in[i].split()
    if (len(name) >= 2) and ( not '.' in name[0]) :
        name_out = name[0] + ' ' + name[-1]
        (name[0] + ' ' + name[-1])
        name_out = unicodedata.normalize('NFKD', name_out).encode('ascii','ignore')
        name_map[name_out] = names_in[i]
df = pd.read_csv('Name_Probs.csv')
df['Og_artist_name'] = df.artist_name.apply(lambda x: name_map[x])
df.to_csv('Name_Probs.csv', encoding='utf-8')

In [ ]:
df.head()

In [ ]:
#create single csv for all big query data
df1 = pd.read_csv('name_freq1.csv')
df2 = pd.read_csv('name_freq2.csv')
df = pd.concat([df1,df2]).drop_duplicates()
df.rename(columns = {'f0_':'first_name_freq'}, inplace = True)
df.to_csv('name_freq.csv')

In [ ]:
len(df)